In [1]:
import pennylane as qml
from pennylane import numpy as np

A quantum graph neural network can be implemented by introducing a Hamiltonian that incorporates information about both nodes and edges. Specifically, single-qubit rotations can be employed to embed node features, while two-qubit operators can be utilized to embed edge features. In the simplest case, we can introduce an Ising model Hamiltonian, where an edge exists between neighboring nodes.

In [2]:
dev = qml.device("default.qubit", wires=4)
n_wires = 4
n_layers = 5

graph = [(0, 1), (0, 3), (1, 2), (2, 3)]

# unitary operator U_B with parameter beta
def Nodes(x):
    for wire in range(n_wires):
        qml.RX(x[wire], wires=wire)


# unitary operator U_C with parameter gamma
def Edges(g,graph):
    for edge in graph:
        wire1 = edge[0]
        wire2 = edge[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(g[wire1, wire2], wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

@qml.qnode(dev)
def Task_4(X,G,graph):

    for wire in range(n_wires):
        qml.Hadamard(wires=wire)

    for i in range(n_layers):
        Edges(G[i],graph)
        Nodes(X[i])

    return qml.expval(qml.PauliZ(0))

In [3]:
X = np.random.uniform(size = (n_layers,n_wires))
G = np.random.uniform(size = (n_layers,n_wires,n_wires))
Net = Task_4(X,G,graph)
circuit1_1 = qml.draw(Task_4)(X,G,graph)
print("\nCircuit1:")
print(circuit1_1)


Circuit1:
0: ──H─╭●───────────╭●─╭●───────────╭●──RX(0.66)────────────────────────╭●─────────────────
1: ──H─╰X──RZ(0.98)─╰X─│────────────│──╭●──────────────────╭●──RX(0.79)─╰X─────────RZ(1.00)
2: ──H─────────────────│────────────│──╰X─────────RZ(0.20)─╰X─╭●──────────────────╭●───────
3: ──H─────────────────╰X──RZ(0.35)─╰X────────────────────────╰X─────────RZ(0.88)─╰X───────

──╭●────────╭●───────────╭●──RX(0.06)────────────────────────╭●──────────────────╭●────────╭●
──╰X────────│────────────│──╭●──────────────────╭●──RX(0.46)─╰X─────────RZ(0.01)─╰X────────│─
───RX(0.26)─│────────────│──╰X─────────RZ(0.81)─╰X─╭●──────────────────╭●─────────RX(0.70)─│─
───RX(0.55)─╰X──RZ(0.24)─╰X────────────────────────╰X─────────RZ(0.68)─╰X─────────RX(0.91)─╰X

────────────╭●──RX(0.43)────────────────────────╭●──────────────────╭●────────╭●───────────╭●
────────────│──╭●──────────────────╭●──RX(0.82)─╰X─────────RZ(0.94)─╰X────────│────────────│─
────────────│──╰X─────────RZ(0.06)─╰X─╭●───────────────